In [1]:
from bs4 import BeautifulSoup     #网页解析，获取数据\n",
import re       #正则表达式，进行文字匹配\n",
import urllib.request,urllib.error      #制定URL，获取网页数据\n",
# import xlwt     #进行excel操作\n",
import sqlite3  #进行SQLite数据库操作\n",
import pandas as pd #存储数据及处理数据\n",
import numpy as np
import requests
import random
import time
from collections import defaultdict
from collections import deque
from collections import Counter
import collections
import json
import sys
import datetime
import os
import csv
from math import *
import pymysql
# from geopy.distance import geodesic
# from pyproj import Proj
pd.set_option("display.max_columns", 100)

from scipy.interpolate import interp1d

# 专利图谱数据

In [14]:
# 读取专利图谱数据
df_link=pd.read_csv(r'E:\project_basedata\2023082103ipc专利关联数据\link_patent_big_ind.csv',dtype='str')
df_nodes=pd.read_csv(r'E:\project_basedata\2023082103ipc专利关联数据\nodes_patent_big_ind.csv',dtype='str')

In [15]:
df_link=df_link.astype(dtype={'Weight':'float'})

# 读取地方专利基础数据：专利数量集聚度高的行业大类

In [7]:
# 读取图谱模拟基底数据，地方优势专利数量所在的行业
df_base=pd.read_csv(r'E:\project_项目\49苏州\专利.csv',dtype='str')
df_base=df_base.astype(dtype={'集聚度':'float'})

# 地方优势专利图谱绘制

In [33]:
df_base_best=df_base.loc[df_base['集聚度']>1]
ls_base_best=df_base_best['行业代码'].unique().tolist()
ls_base_best_1=[x[:3] for x in ls_base_best]

df_nodes.loc[df_nodes['大类'].isin(ls_base_best_1),'type_local']='local_best'
df_nodes.loc[df_nodes['type_local'].isnull(),'type_local']='none'

# 地方专利模拟图谱绘制

In [19]:
def simulate(code_ls,df_net_new,threshold):
    _district_industryCOs=code_ls
    _simulate_list = []
    _simulate_list.append(_district_industryCOs)
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast = []
    while flag == 1:
        _exist_list = []
        for k in range(0,len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []
        for j in range(len(_simulate_list[len(_simulate_list)-1])):
            _industryCO = _simulate_list[len(_simulate_list)-1][j]
            #if _industryCO in industryCOs_product:
            # 从每一个优势产业开始向外找，一步联系值超过的节点
            _df_industryCO_proximity = df_net_new.loc[((df_net_new['Source']==_industryCO) | (df_net_new['Target']==_industryCO))&(df_net_new['Weight']>=threshold)]
            if len(_df_industryCO_proximity)>0:
                _industryCOs_1 = _df_industryCO_proximity.loc[:,'Source'].unique()
                _industryCOs_2 = _df_industryCO_proximity.loc[:,'Target'].unique()

                for m in range(len(_industryCOs_1)):
                    _industryCO_ = _industryCOs_1[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                        _possible_list.append(_industryCO_)
                        ls_a=[]
                        ls_a.append(t)
                        ls_a.append(_industryCO)
                        ls_a.append(_industryCO_)
                        ls_a.append('上游')
                        ls_forcast.append(ls_a)
                        #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        #f.write(ss)
                        c= c+1

                for m in range(len(_industryCOs_2)):
                    _industryCO_ = _industryCOs_2[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                        _possible_list.append(_industryCO_)
                        ls_a=[]
                        ls_a.append(t)
                        ls_a.append(_industryCO)
                        ls_a.append(_industryCO_)
                        ls_a.append('下游')
                        ls_forcast.append(ls_a)
                        #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        #f.write(ss)
                        c= c+1
        if c == 0:
            flag=0
        else:
            flag = 1
            t = t + 1
            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast

# 图谱起点

In [34]:
ls_base_best_2=df_nodes.loc[df_nodes['大类'].isin(ls_base_best_1)]['Id'].unique().tolist()
threshold_medium = 0
# threshold_medium=df_link['Weight'].quantile(0.3)

In [36]:
df_nodes_1=pd.read_excel(r'E:\project_项目\49苏州\predict_patent_nodes.xlsx',dtype='str')
ls_base_best_2=df_nodes_1.loc[df_nodes_1['type_local']=='local_best']['Id'].unique().tolist()
threshold_medium = 0

# 图谱模拟

In [38]:
ls_forcast=simulate(ls_base_best_2,df_link,threshold_medium)
df_forcast=pd.DataFrame(ls_forcast,columns=['t','o','d','direction'])

df_nodes_1.loc[df_nodes_1['Id'].isin(ls_base_best_2),'type2']='local_best'
df_nodes_1.loc[df_nodes_1['type2'].isnull(),'type2']='none'

step_1=df_forcast.loc[(df_forcast['t']==1)]['d'].unique().tolist()
step_2=df_forcast.loc[(df_forcast['t']==2)]['d'].unique().tolist()

df_nodes_1.loc[df_nodes_1['Id'].isin(step_1),'type2']='for1'
df_nodes_1.loc[df_nodes_1['Id'].isin(step_2),'type2']='for2'

df_nodes_1.to_csv(r'E:\project_项目\49苏州\predict_patent_nodes_1.csv',index=False)

In [22]:
# 专利数据小类联系
df_pa_small=pd.read_csv(r'E:\workfile\WXWork\1688851162995685\Cache\File\2023-07\od分组4.csv',dtype='str')
df_pa_small=df_pa_small.rename(columns={'主分类号O':'Source','主分类号D':'Target','Unnamed: 0':'Weight'})
df_pa_small_=df_pa_small.groupby(['Source','Target'])['Weight'].count().reset_index()
# df_pa_small.loc[:,'Weight']=1
# 专利中类联系
df_pa_medium=pd.read_csv(r'E:\project_data\18changsha\patent\medium_graph.csv')

# 专利中类节点
df_pa_medium_nodes=pd.read_csv(r'E:\project_data\18changsha\patent\medium_nodes.csv')

In [24]:
# 图谱起点
df_start_small=pd.read_excel(r'E:\project_data\18changsha\patent\长沙公开专利2013-2022.xlsx',sheet_name='Sheet2')
df_start_small.loc[:,'medium']=df_start_small['行标签'].str[:4]
df_start_medium=df_start_small.groupby(['medium'])['计数项:IPC主分类'].sum().reset_index()

In [25]:
threshold_medium = df_pa_medium['Weight'].quantile(0.3)
threshold_small=0

code_ls_small=df_start_small['行标签'].unique().tolist()
code_ls_medium=df_start_medium['medium'].unique().tolist()
print(len(code_ls_small),len(code_ls_medium))

20983 583


In [38]:
ls_forcast_small=simulate(code_ls_small,df_pa_small_,threshold_small)
df_forcast_small=pd.DataFrame(ls_forcast_small,columns=['t','o','d','direction'])

In [39]:
df_forcast_small.to_csv(r'E:\project_data\18changsha\patent\small_graph.csv',index=False)

In [28]:
ls_forcast_medium=simulate(code_ls_medium,df_pa_medium,threshold_medium)
df_forcast_medium=pd.DataFrame(ls_forcast_medium,columns=['t','o','d','direction'])

In [51]:
df_start_small

,行标签,计数项:IPC主分类,medium
0,C02F9/14,609,C02F
1,F42B4/30,479,F42B
2,C02F9/04,475,C02F
3,A24F47/00,431,A24F
4,H01M4/36,425,H01M
...,...,...,...
20978,A23B7/155,1,A23B
20979,A01N65/10,1,A01N
20980,A23L3/015,1,A23L
20981,A01N47/24,1,A01N


In [72]:
df_start_small_fil=df_start_small.loc[df_start_small['计数项:IPC主分类']>=10]
ls_start=df_start_small_fil['行标签'].unique().tolist()
df_forcast_small_=pd.read_csv(r'E:\project_data\18changsha\patent\small_graph.csv')
#df_forcast_small_graph=df_forcast_small.rename(columns={'o':'Source','d':'Target'})
#df_forcast_small_graph.loc[:,'Weight']=1
df_forcast_small=df_forcast_small_.loc[(df_forcast_small_['t']==1)&(df_forcast_small_['o'].isin(ls_start))]
df_forcast_small.loc[df_forcast_small['direction']=='上游','Source']=df_forcast_small['d']
df_forcast_small.loc[df_forcast_small['direction']=='下游','Source']=df_forcast_small['o']
df_forcast_small.loc[df_forcast_small['direction']=='上游','Target']=df_forcast_small['o']
df_forcast_small.loc[df_forcast_small['direction']=='下游','Target']=df_forcast_small['d']
df_forcast_small_graph1=df_forcast_small.loc[:,['Source','Target']]
df_forcast_small_graph1.loc[:,'Weight']=1
df_forcast_small_graph1.to_csv(r'E:\project_data\18changsha\patent\small_graph_t1.csv',index=False)

ls1=df_forcast_small_graph1['Source'].unique().tolist()
ls2=df_forcast_small_graph1['Target'].unique().tolist()
ls=list(set(ls1+ls2))

df_base_nodes=pd.read_excel(r'E:\project_basedata\2023082103ipc专利关联数据\ipc_new.xlsx')
df_base_nodes=df_base_nodes.drop_duplicates(['code'])
df_base_nodes_1=df_base_nodes.loc[df_base_nodes['name'].notnull()]
df_base_nodes_2=df_base_nodes_1.loc[df_base_nodes_1['code'].isin(ls)]
df_base_nodes_3=df_base_nodes_2.rename(columns={'code':'Id','name':'Label'})
df_base_nodes_3.loc[:,'type0']=df_base_nodes_3['Id'].str[0]
df_base_nodes_3.loc[:,'type1']=df_base_nodes_3['Id'].str[:3]
df_base_nodes_3.loc[:,'type2']=df_base_nodes_3['Id'].str[:4]
df_base_nodes_4=pd.merge(left=df_base_nodes_3,right=df_start_small_fil,left_on='Id',right_on='行标签',how='left')
df_base_nodes_4.loc[df_base_nodes_4['行标签'].notnull(),'type_for0']='best'
df_base_nodes_4.loc[df_base_nodes_4['行标签'].isnull(),'type_for0']=df_base_nodes_4['type0']
df_base_nodes_4.drop(columns=['行标签','medium'],inplace=True)
df_base_nodes_4.loc[df_base_nodes_4['计数项:IPC主分类'].isnull(),'计数项:IPC主分类']=0
df_base_nodes_4.to_csv(r'E:\project_data\18changsha\patent\small_nodes_t1.csv',index=False)

C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_21348\3403849313.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forcast_small.loc[df_forcast_small['direction']=='上游','Source']=df_forcast_small['d']
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_21348\3403849313.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forcast_small.loc[df_forcast_small['direction']=='上游','Target']=df_forcast_small['o']


In [71]:
df_base_nodes.drop_duplicates(['code'])

,code,name
0,A,人类生活必需
1,B,作业；运输
2,C,化学；冶金
3,D,纺织；造纸
4,E,固定建筑物
...,...,...
76815,H05K7/20,便于冷却、通风或加热的改进
76816,H05K9/00,设备或元件对电场或磁场的屏蔽
76817,H99,本部中其他类目不包括的技术主题〔8〕
76818,H99Z,本部其他类目不包括的技术主题〔8〕


In [59]:
df_base_nodes_2=df_base_nodes_1.loc[df_base_nodes_1['code'].isin(ls)]
df_base_nodes_3=df_base_nodes_2.rename(columns={'code':'Id','name':'Label'})
df_base_nodes_3.loc[:,'type0']=df_base_nodes_3['Id'].str[0]
df_base_nodes_3.loc[:,'type1']=df_base_nodes_3['Id'].str[:3]
df_base_nodes_3.loc[:,'type2']=df_base_nodes_3['Id'].str[:4]
df_base_nodes_4=pd.merge(left=df_base_nodes_3,right=df_start_small_fil,left_on='Id',right_on='行标签',how='left')
df_base_nodes_4.loc[df_base_nodes_4['行标签'].notnull(),'type_for0']='best'
df_base_nodes_4.loc[df_base_nodes_4['行标签'].isnull(),'type_for0']=df_base_nodes_4['type0']
df_base_nodes_4.drop(columns=['行标签','medium'],inplace=True)
df_base_nodes_4.loc[df_base_nodes_4['计数项:IPC主分类'].isnull(),'计数项:IPC主分类']=0

In [63]:
df_start_small_fil

,行标签,计数项:IPC主分类,medium
0,C02F9/14,609,C02F
1,F42B4/30,479,F42B
2,C02F9/04,475,C02F
3,A24F47/00,431,A24F
4,H01M4/36,425,H01M
...,...,...,...
3434,A21C3/02,10,A21C
3435,A24C5/00,10,A24C
3436,A01K11/00,10,A01K
3437,A23N12/06,10,A23N


In [67]:
df_base_nodes_3=df_base_nodes_2.rename(columns={'code':'Id','name':'Label'})
df_base_nodes_3.loc[:,'type0']=df_base_nodes_3['Id'].str[0]
df_base_nodes_3.loc[:,'type1']=df_base_nodes_3['Id'].str[:3]
df_base_nodes_3.loc[:,'type2']=df_base_nodes_3['Id'].str[:4]
df_base_nodes_4=pd.merge(left=df_base_nodes_3,right=df_start_small_fil,left_on='Id',right_on='行标签',how='left')
df_base_nodes_4.loc[df_base_nodes_4['行标签'].notnull(),'type_for0']='best'
df_base_nodes_4.loc[df_base_nodes_4['行标签'].isnull(),'type_for0']=df_base_nodes_4['type0']
df_base_nodes_4.drop(columns=['行标签','medium'],inplace=True)
df_base_nodes_4.loc[df_base_nodes_4['计数项:IPC主分类'].isnull(),'计数项:IPC主分类']=0

In [68]:
df_base_nodes_4

,Id,Label,type0,type1,type2,计数项:IPC主分类,type_for0
0,A01B1/00,手动工具,A,A01,A01B,0.0,A
1,A01B1/02,锹；铲,A,A01,A01B,0.0,A
2,A01B1/04,带齿的,A,A01,A01B,0.0,A
3,A01B1/14,只带齿的,A,A01,A01B,0.0,A
4,A01B11/00,带有振动、挖掘或穿孔工作部件的犁,A,A01,A01B,0.0,A
...,...,...,...,...,...,...,...
31371,H05K7/10,组件的插入式组装,H,H05,H05K,0.0,H
31372,H05K7/14,在外壳内或在框架上或在导轨上安装支承结构,H,H05,H05K,68.0,best
31373,H05K7/18,导轨或框架的结构,H,H05,H05K,12.0,best
31374,H05K7/20,便于冷却、通风或加热的改进,H,H05,H05K,253.0,best


In [37]:
ls_best=df_forcast_medium['o'].unique().tolist()
ls_for1=df_forcast_medium['d'].unique().tolist()
df_pa_medium_nodes1=df_pa_medium_nodes.loc[:,['Id','Label','部','大类']]
df_pa_medium_nodes1.loc[df_pa_medium_nodes1['Id'].isin(ls_best),'type']='best'
df_pa_medium_nodes1.loc[df_pa_medium_nodes1['Id'].isin(ls_for1),'type']='for1'
df_pa_medium_nodes1.to_csv(r'E:\project_data\18changsha\patent\medium_nodes_for.csv',index=False)

In [16]:
df_pa_small=df_pa_small.rename(columns={'主分类号O':'Source','主分类号D':'Target'})
df_pa_small.loc[:,'Weight']=1

,Unnamed: 0,公开号O,主分类号O,公开号D,主分类号D,对应O,IPC_O,对应D,IPC_D
0,0,CN103251844B,A61K36/899,CN101690584A,A23L1/30,A61K36/899,禾本科，例如芦根、竹叶、玉蜀黍,A23L1/30,NaN
1,1,CN103584076B,A23L33/00,CN101690584A,A23L1/30,A23L33/00,改变食品的营养性质；营养制品；其制备或处理〔2016.01〕,A23L1/30,NaN
2,2,CN104435400B,A61K36/87,CN101690584A,A23L1/30,A61K36/87,葡萄科，例如酒葡萄，圆叶葡萄或辣葡萄〔８〕,A23L1/30,NaN
3,3,CN103677082B,G06F1/14,CN101692607A,H03K21/00,G06F1/14,时间监控设备，例如，实时时钟〔5〕,H03K21/00,脉冲计数器或分频器的零部件
4,4,CN102880220B,G05F1/567,CN101692607A,H03K21/00,G05F1/567,用于温度补偿的〔4〕,H03K21/00,脉冲计数器或分频器的零部件
...,...,...,...,...,...,...,...,...,...
5252098,5252098,CN103340908A,A61K36/068,CN1055967C,C12N1/14,A61K36/068,虫草属〔８〕,C12N1/14,真菌
5252099,5252099,CN100496537C,A61K36/634,CN1047086C,A61K35/78,A61K36/634,连翘属〔８〕,A61K35/78,NaN
5252100,5252100,CN100564202C,B65G37/00,CN1034374C,H01R43/048,B65G37/00,与在特殊机械中应用或在特殊制造过程中使用无关的相同或不同种类的机械输送机的联合装置,H01R43/048,卷缩设备或方法
5252101,5252101,CN101941064A,B22D13/00,CN1034294C,F16B37/00,B22D13/00,离心铸造；利用离心力的铸造,F16B37/00,螺母或类似的螺纹接合元件


In [15]:
df_pa_medium

,Source,Target,Type,Id,Label,timeset,Weight
0,A01B,A01C,Undirected,1091,NaN,NaN,391.277870
1,A01G,A01D,Undirected,1092,NaN,NaN,170.000000
2,A01D,A01F,Undirected,1093,NaN,NaN,273.472318
3,A01C,A01G,Undirected,1094,NaN,NaN,977.392685
4,A01G,A01H,Undirected,1095,NaN,NaN,904.235320
...,...,...,...,...,...,...,...
1086,A24F,A24D,Undirected,2177,NaN,NaN,0.145800
1087,B27K,B27J,Undirected,2178,NaN,NaN,0.145038
1088,F27D,C21B,Undirected,2179,NaN,NaN,0.144965
1089,B27L,B27G,Undirected,2180,NaN,NaN,0.144928


In [6]:
df_pa_small

,Unnamed: 0,公开号O,主分类号O,公开号D,主分类号D,对应O,IPC_O,对应D,IPC_D
0,0,CN103251844B,A61K36/899,CN101690584A,A23L1/30,A61K36/899,禾本科，例如芦根、竹叶、玉蜀黍,A23L1/30,NaN
1,1,CN103584076B,A23L33/00,CN101690584A,A23L1/30,A23L33/00,改变食品的营养性质；营养制品；其制备或处理〔2016.01〕,A23L1/30,NaN
2,2,CN104435400B,A61K36/87,CN101690584A,A23L1/30,A61K36/87,葡萄科，例如酒葡萄，圆叶葡萄或辣葡萄〔８〕,A23L1/30,NaN
3,3,CN103677082B,G06F1/14,CN101692607A,H03K21/00,G06F1/14,时间监控设备，例如，实时时钟〔5〕,H03K21/00,脉冲计数器或分频器的零部件
4,4,CN102880220B,G05F1/567,CN101692607A,H03K21/00,G05F1/567,用于温度补偿的〔4〕,H03K21/00,脉冲计数器或分频器的零部件
...,...,...,...,...,...,...,...,...,...
5252098,5252098,CN103340908A,A61K36/068,CN1055967C,C12N1/14,A61K36/068,虫草属〔８〕,C12N1/14,真菌
5252099,5252099,CN100496537C,A61K36/634,CN1047086C,A61K35/78,A61K36/634,连翘属〔８〕,A61K35/78,NaN
5252100,5252100,CN100564202C,B65G37/00,CN1034374C,H01R43/048,B65G37/00,与在特殊机械中应用或在特殊制造过程中使用无关的相同或不同种类的机械输送机的联合装置,H01R43/048,卷缩设备或方法
5252101,5252101,CN101941064A,B22D13/00,CN1034294C,F16B37/00,B22D13/00,离心铸造；利用离心力的铸造,F16B37/00,螺母或类似的螺纹接合元件


In [7]:
df_pa_medium

,Source,Target,Type,Id,Label,timeset,Weight
0,A01B,A01C,Undirected,1091,NaN,NaN,391.277870
1,A01G,A01D,Undirected,1092,NaN,NaN,170.000000
2,A01D,A01F,Undirected,1093,NaN,NaN,273.472318
3,A01C,A01G,Undirected,1094,NaN,NaN,977.392685
4,A01G,A01H,Undirected,1095,NaN,NaN,904.235320
...,...,...,...,...,...,...,...
1086,A24F,A24D,Undirected,2177,NaN,NaN,0.145800
1087,B27K,B27J,Undirected,2178,NaN,NaN,0.145038
1088,F27D,C21B,Undirected,2179,NaN,NaN,0.144965
1089,B27L,B27G,Undirected,2180,NaN,NaN,0.144928


In [14]:
df_start_medium

,medium,计数项:IPC主分类
0,A01B,181
1,A01C,366
2,A01D,210
3,A01F,77
4,A01G,1226
...,...,...
578,H05B,189
579,H05F,7
580,H05G,1
581,H05H,18
